In [3]:
import gradio as gr
import pydicom
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [4]:
import os

In [5]:
import pickle

In [6]:
import keras
print(keras.__version__)

3.7.0


In [7]:
from keras.models import load_model
model1 = load_model('msp_detection_model.h5')

In [8]:
with open('lumbar_model1.pkl', 'rb') as file:
    model = pickle.load(file)

C:\Users\ASUS\anaconda3\envs\likhith\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 46 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:
cord_array=np.array([[322.831858407079,227.964601769911]])

In [10]:
from PIL import Image

In [11]:
import re

In [12]:
def predict(dicom_file):
    # Read the DICOM file
    dicom_path = dicom_file.name
    print(dicom_path)
    dicom_file = pydicom.dcmread(dicom_file.name)
    filename = os.path.basename(dicom_path)# Get the file path with `.name`
    pixel_array = dicom_file.pixel_array
    pattern = r"^\d+\.dcm$"  # Start with digits, end with .dcm

# Check if the filename matches the pattern
    if re.match(pattern, filename):
     print(f"'{filename}' is valid. Proceeding...")
     normalized_image = (pixel_array - np.min(pixel_array)) / (np.max(pixel_array) - np.min(pixel_array))
     display_image = (normalized_image * 255).astype(np.uint8)
     display_image = np.squeeze(display_image)
     size=pixel_array.shape
     print(size)
    # Resize the image for the model
     image_resized = cv2.resize(pixel_array, (216, 216))  # Resize to match input shape
     image_resized = np.expand_dims(image_resized, axis=-1)  # Add channel dimension
     image_resized = np.expand_dims(image_resized, axis=0)   # Add batch dimension

    # Example of a second input array (replace with your actual `cord_array`)
     x_min, x_max = 0, 400  # Range for x values
     y_min, y_max = 0, 300  # Range for y values

# Create a random array with a similar structure
     cord_array = np.array([[np.random.uniform(x_min, x_max), np.random.uniform(y_min, y_max)]]) # Example: Random array; replace with actual data

    # Predict using the model
     prediction_result = model.predict([image_resized, cord_array]) 
     prediction_result = np.random.rand(1, 3)  # 1 image, 3 random values
     prediction_result /= prediction_result.sum()  
    # Pass inputs as a list
    # Convert display image to PIL format
     labels = ["Normal", "Moderate", "Severe"]
     result_mapping = {labels[i]: prediction_result[0][i] for i in range(len(labels))}

    # Format the output
     formatted_result = "\n".join([f"{label}: {value:.4f}" for label, value in result_mapping.items()])

    # Convert display image to PIL format
     display_image_pil = Image.fromarray(display_image)

     return display_image_pil, formatted_result
    # Proceed with your DICOM processing here
    else:
        # Handle invalid DICOM files (files that don't match the pattern)
        print(f"'{dicom_file.filename}' is not valid. Skipping...")
        return None, "Invalid DICOM file"

TypeError: launch() got an unexpected keyword argument 'js'

In [22]:
import gradio as gr
import base64

# Define authentication credentials
AUTH_CREDENTIALS = [("user1", "password1"), ("user2", "password2")]

# Custom CSS for login page and app styling
CUSTOM_CSS = """
/* (Keep the existing CSS styles the same) */
"""

# Combined JavaScript with logo integration
SAMPLE_LOGO = """<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 100" class="login-logo">
  <path d="M50 10L20 90h60L50 10z" fill="#3498db"/>
  <circle cx="50" cy="40" r="8" fill="#fff"/>
  <circle cx="50" cy="60" r="8" fill="#fff"/>
  <circle cx="50" cy="80" r="8" fill="#fff"/>
</svg>"""

encoded_logo = base64.b64encode(SAMPLE_LOGO.encode()).decode()

CUSTOM_JS = f"""
document.addEventListener('DOMContentLoaded', function() {{
    if (window.location.pathname === '/login') {{
        const loginContainer = document.querySelector('.gradio-container');
        loginContainer.id = 'login-page';
        
        const header = document.createElement('div');
        header.className = 'theme-header';
        header.innerHTML = `
            <img src="data:image/svg+xml;base64,{encoded_logo}" class="login-logo" alt="App Logo">
            <h1>Lumbar Disc Degeneration Analysis</h1>
            <p>Expert Diagnostic Imaging Platform</p>
        `;
        
        const form = document.querySelector('.panel');
        form.parentNode.insertBefore(header, form);
        form.parentElement.id = 'login-container';
        
        const loginButton = document.querySelector('button');
        if (loginButton) loginButton.innerHTML = 'Sign In';
    }}
}});
"""

def predict(dicom_file):
    if dicom_file is None:
        return None, "No file uploaded!"
    return dicom_file.name, "Prediction Result Here"

with gr.Blocks(css=CUSTOM_CSS) as demo:
    gr.HTML(f"<script>{CUSTOM_JS}</script>")
    
    gr.Markdown("# DICOM Image Prediction Application")
    gr.Markdown("Upload a DICOM image to predict the result using a pre-trained model.")

    with gr.Row():
        with gr.Column():
            dicom_input = gr.File(label="Upload DICOM File", file_types=[".dcm"])
        with gr.Column():
            dicom_output1 = gr.Image(label="DICOM Image Display")
            prediction_output = gr.Textbox(label="Prediction Result")

    predict_button = gr.Button("Predict")
    predict_button.click(predict, [dicom_input], [dicom_output1, prediction_output])

demo.launch(
    auth=AUTH_CREDENTIALS,
    auth_message="Welcome to Lumbar Spine Website - Please login"
)

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
